In [4]:
import boto3
import os
from dotenv import load_dotenv
from pathlib import Path

# --- BAGIAN PERBAIKAN PATH ---
# Kita cari file .env secara manual biar gak salah alamat
current_dir = Path(os.getcwd())
possible_paths = [
    current_dir / ".env",          # Cek di folder yang sama dengan notebook
    current_dir.parent / ".env",   # Cek di folder luar (Kulit.ai/)
    Path("../.env")                # Cek pake relative path standar
]

env_found = False
for path in possible_paths:
    if path.exists():
        load_dotenv(path)
        print(f"✅ File .env ditemukan di: {path}")
        env_found = True
        break

if not env_found:
    print("❌ GAWAT! File .env TIDAK DITEMUKAN di manapun!")
    print("👉 Pastikan kamu sudah bikin file bernama '.env' di folder 'Kulit.ai'")
    # --- EMERGENCY: Kalau kepepet, buka comment di bawah dan isi manual ---
    # os.environ["AWS_ACCESS_KEY_ID"] = "AKIA.....(Isi Punya Kamu)"
    # os.environ["AWS_SECRET_ACCESS_KEY"] = "abcde....(Isi Punya Kamu)"
    # os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

# Cek apakah credential terbaca
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
region = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')

print(f"🔑 Access Key Loaded: {bool(access_key)} (Depan: {access_key[:4] if access_key else 'None'}...)")
print(f"🌎 Region: {region}")

if not access_key or not secret_key:
    raise ValueError("⛔ STOP! Credential belum terisi. Cek file .env kamu!")

# --- KONFIGURASI ---
BUCKET_NAME = "kulit-ai-dataset-real-fiki-2025"  # Pastikan nama ini unik
REGION = region

s3 = boto3.client('s3', region_name=REGION)

def create_bucket():
    """Membuat S3 Bucket jika belum ada"""
    try:
        if REGION == "us-east-1":
            s3.create_bucket(Bucket=BUCKET_NAME)
        else:
            s3.create_bucket(
                Bucket=BUCKET_NAME,
                CreateBucketConfiguration={'LocationConstraint': REGION}
            )
        print(f"✅ Bucket siap: {BUCKET_NAME}")
    except Exception as e:
        # 409 = BucketAlreadyOwnedByYou (Aman, lanjut aja)
        if "BucketAlreadyOwnedByYou" in str(e):
            print(f"✅ Bucket sudah ada (Milikmu): {BUCKET_NAME}")
        else:
            print(f"ℹ️ Info Bucket: {e}")

def upload_folder(local_path, s3_prefix):
    """Upload isi folder lokal ke S3"""
    # Fix path agar absolut (mencegah error 'not found')
    abs_path = os.path.abspath(local_path)
    
    if not os.path.exists(abs_path):
        print(f"⚠️ Warning: Folder {abs_path} tidak ditemukan! (Cek nama folder data kamu)")
        return

    print(f"📂 Mengupload dari: {abs_path} \n   --> ke S3://{BUCKET_NAME}/{s3_prefix}")
    
    file_count = 0
    for root, dirs, files in os.walk(abs_path):
        for file in files:
            local_file = os.path.join(root, file)
            relative_path = os.path.relpath(local_file, abs_path)
            s3_path = os.path.join(s3_prefix, relative_path).replace("\\", "/")
            
            s3.upload_file(local_file, BUCKET_NAME, s3_path)
            file_count += 1
            if file_count % 50 == 0: 
                print(f"   ⬆️ Uploaded {file_count} files...")
    
    print(f"✅ Selesai upload {s3_prefix} ({file_count} files)")

# --- EKSEKUSI ---
if __name__ == "__main__":
    create_bucket()
    # Kita gunakan path relative standar (asumsi folder data sejajar dengan folder notebooks)
    # Gunakan '../data' jika notebook dijalankan dari dalam folder notebooks
    upload_folder("../data/train", "data/train")
    upload_folder("../data/test", "data/test")
    upload_folder("../data/valid", "data/valid")
    
    print("\n🎉 SEMUA DATA SUDAH DI CLOUD! SIAP UNTUK SAGE MAKER!")

✅ File .env ditemukan di: d:\AI_ML ENGINEER\Kulit.ai\.env
🔑 Access Key Loaded: True (Depan: AKIA...)
🌎 Region: us-east-1
✅ Bucket siap: kulit-ai-dataset-real-fiki-2025
📂 Mengupload dari: d:\AI_ML ENGINEER\Kulit.ai\data\train 
   --> ke S3://kulit-ai-dataset-real-fiki-2025/data/train
   ⬆️ Uploaded 50 files...
   ⬆️ Uploaded 100 files...
   ⬆️ Uploaded 150 files...
   ⬆️ Uploaded 200 files...
   ⬆️ Uploaded 250 files...
   ⬆️ Uploaded 300 files...
   ⬆️ Uploaded 350 files...
   ⬆️ Uploaded 400 files...
   ⬆️ Uploaded 450 files...
   ⬆️ Uploaded 500 files...
   ⬆️ Uploaded 550 files...
   ⬆️ Uploaded 600 files...
   ⬆️ Uploaded 650 files...
   ⬆️ Uploaded 700 files...
   ⬆️ Uploaded 750 files...
   ⬆️ Uploaded 800 files...
   ⬆️ Uploaded 850 files...
   ⬆️ Uploaded 900 files...
   ⬆️ Uploaded 950 files...
   ⬆️ Uploaded 1000 files...
   ⬆️ Uploaded 1050 files...
   ⬆️ Uploaded 1100 files...
   ⬆️ Uploaded 1150 files...
   ⬆️ Uploaded 1200 files...
   ⬆️ Uploaded 1250 files...
   ⬆️ Uploa